In [1]:
import os
import shutil
import pickle
import numpy as np
import torch
import copy
import torch.nn as nn

from skopt.optimizer import gp_minimize
from skopt.space import Real, Categorical, Integer

from bcnf.simulation.physics import get_data
from bcnf.models.cnf import CondRealNVP
from bcnf.models.feature_network import FullyConnectedFeatureNetwork
from bcnf.eval.crossvalidate import cross_validate
from bcnf.errors import TrainingDivergedError

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
np.int = np.int64

In [3]:
# Check if a checkpoint exists
checkpoint_file = 'checkpoint.pkl'

In [4]:
X, y = get_data(
    T=5.0,
    dt=0.1,
    N=2_500,
    break_on_impact=False
)

X_tensor = torch.Tensor(X.reshape(X.shape[0], -1))
y_tensor = torch.Tensor(y)

100%|██████████| 2500/2500 [00:04<00:00, 605.38it/s]


In [5]:
optimizer_kwargs = {
    "lr": 1e-4
}

lr_scheduler_kwargs = {
    "mode": "min",
    "factor": 0.8,
    "patience": 200,
}

In [6]:
# Example search spaces
search_spaces = {
    'condition_size': Integer(1, 2048),
    'model_hidden_size': Integer(32, 256),
    'model_n_blocks': Integer(1, 64),
    'model_act_norm': Categorical([True, False]),
    'feature_network_hidden_size': Integer(16, 2048),
    'feature_network_hidden_layers': Integer(0, 16),
    'feature_network_dropout': Real(0, 0.5),
}

In [7]:
def param_index(name, search_spaces):
    return list(search_spaces.keys()).index(name)

In [8]:
model_size = y_tensor.shape[1]
feature_size = X_tensor.shape[1]

In [9]:
def score_parameters(params: list):
    print(params)
    try:
        fold_metrics = cross_validate(
            model_class=CondRealNVP,
            model_kwargs={
                "size": model_size,
                "hidden_size": params[param_index('model_hidden_size', search_spaces)],
                "n_blocks": params[param_index('model_n_blocks', search_spaces)],
                "n_conditions": params[param_index('condition_size', search_spaces)],
                "act_norm": params[param_index('model_act_norm', search_spaces)],
            },
            feature_network_class=FullyConnectedFeatureNetwork,
            feature_network_kwargs={
                "sizes": [feature_size]
                    + [params[param_index('feature_network_hidden_size', search_spaces)]] * params[param_index('feature_network_hidden_layers', search_spaces)]
                    + [params[param_index('condition_size', search_spaces)]],
                "dropout": params[param_index('feature_network_dropout', search_spaces)],    
            },
            optimizer_class=torch.optim.Adam,
            optimizer_kwargs=optimizer_kwargs,
            lr_scheduler_class=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs=lr_scheduler_kwargs,
            X=X_tensor,
            y=y_tensor,
            n_epochs=50_000,
            val_loss_patience=500,
            val_loss_tolerance=1e-4,
            batch_size=256,
            device=device,
            verbose=True,
            n_splits=3,
            errors="raise"
        )
    except TrainingDivergedError as e:
        print(e)
        return 100  # A big number

    val_loss_list = [r['val_loss'] for r in fold_metrics]
    print(f'Val Loss: {np.mean(val_loss_list):.4f} ± {np.std(val_loss_list):.4f}')
    return np.mean(val_loss_list) + np.std(val_loss_list)

In [10]:
# Define a callback function to save the results
def save_checkpoint(result):
    # Save the checkpoint after each iteration or at the end of the optimization
    with open(checkpoint_file + ".tmp", 'wb') as f:
        # Ignore
        # - result['specs']['args']['func']
        # - result['specs']['args']['callback']
        # because it causes problems when reading somewhere else
        result_no_func = copy.deepcopy(result)
        del result_no_func['specs']['args']['func']
        del result_no_func['specs']['args']['callback']
        pickle.dump(result_no_func, f)

    # Delete the old checkpoint file and rename the temporary file
    shutil.move(checkpoint_file + ".tmp", checkpoint_file)

In [11]:
N_STEPS = 100

In [12]:
if os.path.exists(checkpoint_file):
    print(f'Loading checkpoint from {checkpoint_file}')
    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)
        checkpoint['specs']['args']['func'] = score_parameters
        checkpoint['specs']['args']['callback'] = save_checkpoint
    print(f'Resuming from iteration {len(checkpoint.x_iters)}')
else:
    print('No checkpoint found. Starting new optimization')
    checkpoint = None

# You might want to adjust the n_calls or other parameters based on the checkpoint
result = gp_minimize(
    func=score_parameters,
    dimensions=search_spaces.values(),
    n_initial_points=10,
    n_calls=N_STEPS - len(checkpoint.x_iters) if checkpoint else N_STEPS,
    random_state=2024_03_25,
    verbose=True,
    callback=save_checkpoint,
    acq_func='EI',
    x0 = checkpoint.x_iters if checkpoint else None,
    y0 = checkpoint.func_vals if checkpoint else None)

No checkpoint found. Starting new optimization
Iteration No: 1 started. Evaluating function at random point.


Train: -8.0675 - Val: -9.9729 (avg: -8.0749, min: -7.9750) | lr: 1.00e-04 - Patience: 1/500:   0%|          | 165/50000 [00:21<1:45:55,  7.84it/s]


KeyboardInterrupt: 

In [ ]:
if os.path.exists(checkpoint_file):
    print(f'Loading checkpoint from {checkpoint_file}')
    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)

Loading checkpoint from checkpoint.pkl


In [ ]:
checkpoint['specs']['args']['callback']

<function __main__.save_checkpoint(result)>